# OpenAI Python with LLM Contract Language (LLMCL) Framework

This notebook demonstrates the comprehensive features of the OpenAI Python SDK enhanced with the LLM Contract Language framework for building reliable, contract-based LLM applications.

## Table of Contents
1. [Introduction and Setup](#1-introduction-and-setup)
2. [Basic OpenAI SDK Usage](#2-basic-openai-sdk-usage)
3. [Contract-Based Validation](#3-contract-based-validation)
4. [LLMCL (LLM Contract Language)](#4-llmcl-llm-contract-language)
5. [Streaming with Contract Validation](#5-streaming-with-contract-validation)
6. [Advanced Features](#6-advanced-features)
7. [Performance Monitoring](#7-performance-monitoring)
8. [Real-World Examples](#8-real-world-examples)

## 1. Introduction and Setup

The LLM Design by Contract Framework provides:
- **Contract-Based Validation**: Input and output validation using design-by-contract principles
- **Multi-Provider Support**: Currently supports OpenAI with extensible architecture
- **Comprehensive Contract Types**: 7 different contract categories
- **Streaming Support**: Real-time validation for streaming responses
- **Auto-Fix Suggestions**: Intelligent suggestions for contract violations
- **LLMCL**: A domain-specific language for defining contracts

In [ ]:
# Installation (run once)
!pip install -e .
!pip install openai

: 

In [ ]:
# Import required libraries
import os
import json
import asyncio
from typing import Dict, Any, List

# OpenAI imports
from openai import OpenAI

# Contract framework imports
from llm_contracts.providers.openai_provider import ImprovedOpenAIProvider
from llm_contracts.contracts.base import (
    PromptLengthContract,
    PromptInjectionContract,
    ContentPolicyContract,
    JSONFormatContract,
    ResponseTimeContract,
    ConversationConsistencyContract,
    MedicalDisclaimerContract
)
from llm_contracts.validators import InputValidator, OutputValidator
from llm_contracts.core.exceptions import ContractViolationError

# LLMCL imports
from llm_contracts.language import LLMCLRuntime, ResolutionStrategy

print("✅ All imports successful!")

In [ ]:
# Set up your OpenAI API key
# Option 1: Set as environment variable
# os.environ["OPENAI_API_KEY"] = "your-api-key-here"

# Option 2: Pass directly (for demo purposes only)
api_key = os.getenv("OPENAI_API_KEY", "your-api-key-here")

if api_key == "your-api-key-here":
    print("⚠️  Please set your OpenAI API key!")
else:
    print("✅ API key configured")

## 2. Basic OpenAI SDK Usage

First, let's see how the standard OpenAI SDK works before adding contracts.

In [ ]:
# Standard OpenAI client
standard_client = OpenAI(api_key=api_key)

# Basic chat completion
response = standard_client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "What is the capital of France?"}
    ],
    temperature=0.7
)

print("Standard OpenAI Response:")
print(response.choices[0].message.content)

## 3. Contract-Based Validation

Now let's enhance the OpenAI client with contract-based validation.

In [ ]:
# Create an improved OpenAI provider with contract support
provider = ImprovedOpenAIProvider(api_key=api_key)

# Add input contracts
provider.add_input_contract(PromptLengthContract(max_tokens=100))
provider.add_input_contract(PromptInjectionContract())

# Add output contracts
provider.add_output_contract(JSONFormatContract())

print("✅ Contract-enhanced provider created with input and output validation")

In [ ]:
# Example 1: Valid request that passes all contracts
try:
    response = provider.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant. Always respond in valid JSON format."},
            {"role": "user", "content": "Generate a user profile with name and age."}
        ]
    )
    print("✅ Response passed all contracts:")
    print(response.choices[0].message.content)
except ContractViolationError as e:
    print(f"❌ Contract violation: {e}")

In [ ]:
# Example 2: Input contract violation (prompt too long)
try:
    long_prompt = "This is a very long prompt. " * 50  # Exceeds token limit
    response = provider.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": long_prompt}
        ]
    )
except ContractViolationError as e:
    print(f"❌ Expected contract violation: {e}")
    print(f"   Suggestion: {e.suggestion if hasattr(e, 'suggestion') else 'N/A'}")

In [ ]:
# Example 3: Different contract types
# Create a provider with multiple contract types
multi_contract_provider = ImprovedOpenAIProvider(api_key=api_key)

# Add various contract types
contracts_demo = {
    "Input Contracts": [
        PromptLengthContract(max_tokens=200),
        ContentPolicyContract(),
        PromptInjectionContract()
    ],
    "Output Contracts": [
        ResponseTimeContract(max_response_time=5.0),
        ConversationConsistencyContract(),
        MedicalDisclaimerContract()
    ]
}

# Add all contracts
for contract_type, contracts in contracts_demo.items():
    print(f"\nAdding {contract_type}:")
    for contract in contracts:
        if "Input" in contract_type:
            multi_contract_provider.add_input_contract(contract)
        else:
            multi_contract_provider.add_output_contract(contract)
        print(f"  - {contract.__class__.__name__}")

print("\n✅ Multiple contracts configured!")

## 4. LLMCL (LLM Contract Language)

LLMCL is a domain-specific language for defining contracts declaratively.

In [ ]:
# Define contracts using LLMCL syntax
safety_contract = """
contract ChatbotSafety(priority = critical) {
    # Input validation
    require len(content) > 0 and len(content) < 4000
        message: "Input must be between 1 and 4000 characters"
    
    require not match(content, "(?i)(injection|exploit)")
        message: "Potential security threat detected"
    
    # Output validation
    ensure not contains(response, "password")
        message: "Response contains sensitive information"
    
    ensure len(response) > 10
        message: "Response too short"
        auto_fix: "I'd be happy to help you. Could you please provide more details?"
}
"""

api_response_contract = """
contract APIResponse {
    # Ensure valid JSON
    ensure json_valid(response)
        message: "Response must be valid JSON"
        auto_fix: '{"error": "Invalid response format", "original": "' + response + '"}'
    
    # Ensure required fields
    ensure contains(response, '"status"') and contains(response, '"data"')
        message: "Response must contain status and data fields"
}
"""

quality_contract = """
contract ConversationQuality(priority = high) {
    # Ensure helpful responses
    ensure len(response) > 50 or contains(response, "?")
        message: "Provide detailed responses or ask clarifying questions"
    
    # Probabilistic quality check
    ensure_prob not startswith(response, "I don't know"), 0.8
        message: "Should provide helpful answers at least 80% of the time"
        window_size: 50
    
    # Temporal constraint
    temporal within 3 contains(response, "help") or contains(response, "assist")
        message: "Offer help within first 3 responses"
}
"""

print("✅ LLMCL contracts defined")

In [ ]:
# Initialize LLMCL runtime
runtime = LLMCLRuntime()

# Load contracts asynchronously
async def load_contracts():
    print("📝 Loading LLMCL contracts...")
    
    safety_name = await runtime.load_contract(safety_contract)
    api_name = await runtime.load_contract(api_response_contract)
    quality_name = await runtime.load_contract(quality_contract)
    
    print(f"✅ Loaded contracts: {safety_name}, {api_name}, {quality_name}")
    return safety_name, api_name, quality_name

# Run the async function
safety_name, api_name, quality_name = await load_contracts()

In [ ]:
# Create runtime context with conflict resolution
context = runtime.create_context(
    "demo_session",
    conflict_strategy=ResolutionStrategy.MOST_RESTRICTIVE
)

# Add contracts to context
runtime.add_contract_to_context("demo_session", safety_name)
runtime.add_contract_to_context("demo_session", quality_name)

print("✅ Runtime context created with contracts")

In [ ]:
# Test LLMCL validation
async def test_llmcl_validation():
    print("🔍 Testing LLMCL Validation\n")
    
    # Test 1: Valid response
    valid_response = "This is a helpful and detailed response about Python programming. " \
                    "It provides useful information without any sensitive data."
    
    result = await runtime.validate(
        valid_response,
        "demo_session",
        validation_type="output",
        additional_context={"content": "Tell me about Python"}
    )
    print(f"1. Valid response: {result.is_valid}")
    
    # Test 2: Too short response
    short_response = "OK"
    result = await runtime.validate(
        short_response,
        "demo_session",
        validation_type="output",
        additional_context={"content": "Explain quantum computing"}
    )
    print(f"\n2. Short response: {result.is_valid}")
    if not result.is_valid:
        print(f"   Violation: {result.message}")
    
    # Test 3: Security violation
    security_violation = "Your password is stored in the database"
    result = await runtime.validate(
        security_violation,
        "demo_session",
        validation_type="output",
        additional_context={"content": "How do I reset my password?"}
    )
    print(f"\n3. Security violation: {result.is_valid}")
    if not result.is_valid:
        print(f"   Violation: {result.message}")
    
    # Test 4: Auto-remediation
    print(f"\n4. Auto-remediation test:")
    print(f"   Original: {short_response}")
    fixed = await runtime.apply_auto_fix(short_response, "demo_session")
    print(f"   Fixed: {fixed}")

await test_llmcl_validation()

In [ ]:
# Test JSON API validation
async def test_json_validation():
    print("📊 Testing JSON API Validation\n")
    
    # Create API context
    api_context = runtime.create_context("api_session")
    runtime.add_contract_to_context("api_session", api_name)
    
    # Test 1: Valid JSON
    valid_json = '{"status": "success", "data": {"id": 123, "name": "test"}}'
    result = await runtime.validate(valid_json, "api_session")
    print(f"1. Valid JSON: {result.is_valid}")
    
    # Test 2: Invalid JSON
    invalid_json = "This is not JSON"
    result = await runtime.validate(invalid_json, "api_session")
    print(f"\n2. Invalid JSON: {result.is_valid}")
    if not result.is_valid:
        print(f"   Violation: {result.message}")
    
    # Test 3: Apply auto-fix
    fixed_json = await runtime.apply_auto_fix(invalid_json, "api_session")
    print(f"   Auto-fixed: {fixed_json}")
    
    # Validate the fixed JSON
    try:
        parsed = json.loads(fixed_json)
        print(f"   ✅ Fixed JSON is valid: {json.dumps(parsed, indent=2)}")
    except:
        print(f"   ❌ Fixed JSON is still invalid")

await test_json_validation()

## 5. Streaming with Contract Validation

The framework supports real-time validation of streaming responses.

In [ ]:
# Create a provider for streaming
stream_provider = ImprovedOpenAIProvider(api_key=api_key)

# Add contracts for streaming validation
stream_provider.add_output_contract(JSONFormatContract())

print("✅ Streaming provider configured")

In [ ]:
# Example: Streaming response with validation
print("🌊 Streaming Response with Contract Validation:\n")

try:
    stream = stream_provider.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant. Always respond in valid JSON format."},
            {"role": "user", "content": "Create a JSON object with weather data including temperature, humidity, and conditions."}
        ],
        stream=True
    )
    
    # Collect streamed chunks
    full_response = ""
    for chunk in stream:
        if chunk.choices[0].delta.content:
            content = chunk.choices[0].delta.content
            full_response += content
            print(content, end="", flush=True)
    
    print("\n\n✅ Streaming completed successfully!")
    
    # Validate the complete response
    try:
        parsed = json.loads(full_response)
        print("✅ Response is valid JSON:")
        print(json.dumps(parsed, indent=2))
    except:
        print("❌ Response is not valid JSON")
        
except ContractViolationError as e:
    print(f"\n❌ Contract violation during streaming: {e}")

## 6. Advanced Features

Let's explore some advanced features of the framework.

In [ ]:
# Feature 1: Circuit Breaker Pattern
# The framework includes a circuit breaker to prevent cascade failures

advanced_provider = ImprovedOpenAIProvider(api_key=api_key)
advanced_provider.add_output_contract(JSONFormatContract())

# The circuit breaker activates after repeated contract failures
print("🔌 Circuit Breaker Pattern:")
print("- Monitors contract validation failures")
print("- Opens circuit after threshold (default: 5 failures)")
print("- Allows degraded operation without validation")
print("- Auto-resets after timeout period")

# You can manually reset if needed
advanced_provider.reset_circuit_breaker()
print("\n✅ Circuit breaker reset")

In [ ]:
# Feature 2: Async Support
async def async_example():
    """Example of async API calls with contracts."""
    print("⚡ Async API Call Example:\n")
    
    # Async provider works the same way
    async_provider = ImprovedOpenAIProvider(api_key=api_key)
    async_provider.add_input_contract(PromptLengthContract(max_tokens=150))
    async_provider.add_output_contract(JSONFormatContract())
    
    # Make async call
    response = await async_provider.chat.completions.acreate(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant. Always respond in JSON."},
            {"role": "user", "content": "Create a JSON object with book information."}
        ]
    )
    
    content = response.choices[0].message.content
    print(f"Response: {content}")
    
    try:
        parsed = json.loads(content)
        print("\n✅ Valid JSON response:")
        print(json.dumps(parsed, indent=2))
    except:
        print("\n❌ Invalid JSON response")

# Run async example
await async_example()

In [ ]:
# Feature 3: Custom Contract Creation
from llm_contracts.contracts.base import BaseContract
from llm_contracts.core.interfaces import ValidationResult

class CustomWordCountContract(BaseContract):
    """Custom contract to ensure response has specific word count."""
    
    def __init__(self, min_words: int = 10, max_words: int = 100):
        super().__init__()
        self.name = "WordCountContract"
        self.min_words = min_words
        self.max_words = max_words
    
    def validate(self, content: str) -> ValidationResult:
        """Validate word count in response."""
        word_count = len(content.split())
        
        if word_count < self.min_words:
            return ValidationResult(
                is_valid=False,
                message=f"Response too short: {word_count} words (minimum: {self.min_words})",
                auto_fix_suggestion="Please provide a more detailed response with additional information."
            )
        
        if word_count > self.max_words:
            return ValidationResult(
                is_valid=False,
                message=f"Response too long: {word_count} words (maximum: {self.max_words})",
                auto_fix_suggestion="Please provide a more concise response."
            )
        
        return ValidationResult(is_valid=True)

# Use custom contract
custom_provider = ImprovedOpenAIProvider(api_key=api_key)
custom_provider.add_output_contract(CustomWordCountContract(min_words=20, max_words=50))

print("✅ Custom contract created and added to provider")

In [ ]:
# Test custom contract
try:
    response = custom_provider.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": "Explain quantum computing in exactly 30 words."}
        ]
    )
    
    content = response.choices[0].message.content
    word_count = len(content.split())
    
    print(f"Response ({word_count} words):")
    print(content)
    print("\n✅ Response passed word count validation!")
    
except ContractViolationError as e:
    print(f"❌ Contract violation: {e}")

## 7. Performance Monitoring

The framework includes comprehensive metrics and performance monitoring.

In [ ]:
# Create a provider and make some calls to generate metrics
metrics_provider = ImprovedOpenAIProvider(api_key=api_key)
metrics_provider.add_input_contract(PromptLengthContract(max_tokens=100))
metrics_provider.add_output_contract(JSONFormatContract())

print("📊 Generating performance metrics...\n")

# Make several calls to generate metrics
for i in range(3):
    try:
        response = metrics_provider.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "Always respond in JSON format."},
                {"role": "user", "content": f"Generate a JSON object with data for item {i+1}"}
            ],
            max_tokens=100
        )
        print(f"✅ Request {i+1} successful")
    except Exception as e:
        print(f"❌ Request {i+1} failed: {e}")

print("\n" + "="*50)

In [ ]:
# Get and display metrics
metrics = metrics_provider.get_metrics()

print("📈 Performance Metrics Report:\n")

print(f"Total Validations: {metrics['total_validations']}")
print(f"Violation Rate: {metrics['violation_rate']:.2%}")

print("\nSlowest Contracts:")
for contract in metrics.get('slowest_contracts', []):
    print(f"  - {contract['name']}: {contract['avg_latency']:.3f}s avg")

print("\nMost Violated Contracts:")
for contract in metrics.get('most_violated_contracts', []):
    print(f"  - {contract['name']}: {contract['violations']} violations")

print("\nAuto-Fix Success Rates:")
for contract_name, stats in metrics.get('auto_fix_success_rates', {}).items():
    if stats['total'] > 0:
        success_rate = stats['success'] / stats['total'] * 100
        print(f"  - {contract_name}: {success_rate:.1f}% ({stats['success']}/{stats['total']})")

In [ ]:
# LLMCL Runtime Metrics
print("\n📊 LLMCL Runtime Metrics:\n")

global_metrics = runtime.get_global_metrics()
for key, value in global_metrics.items():
    print(f"{key}: {value}")

print("\nSession Statistics:")
stats = runtime.get_context_statistics("demo_session")
print(f"Total validations: {stats['total_validations']}")
print(f"Total violations: {stats['total_violations']}")
print(f"Auto-fixes applied: {stats['total_auto_fixes']}")

## 8. Real-World Examples

Let's explore some practical use cases for the contract framework.

In [ ]:
# Example 1: API Response Generation with Strict Schema
print("🌐 Example 1: API Response Generation\n")

# Define a strict JSON schema
api_schema = {
    "type": "object",
    "properties": {
        "status": {"type": "string", "enum": ["success", "error"]},
        "data": {
            "type": "object",
            "properties": {
                "id": {"type": "integer"},
                "name": {"type": "string"},
                "email": {"type": "string", "format": "email"},
                "created_at": {"type": "string", "format": "date-time"}
            },
            "required": ["id", "name", "email"]
        },
        "message": {"type": "string"}
    },
    "required": ["status", "data"]
}

# Create provider with schema validation
api_provider = ImprovedOpenAIProvider(api_key=api_key)
api_provider.add_output_contract(JSONFormatContract(schema=api_schema))

try:
    response = api_provider.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are an API that returns user data in JSON format."},
            {"role": "user", "content": "Get user with ID 123"}
        ]
    )
    
    content = response.choices[0].message.content
    data = json.loads(content)
    
    print("✅ Valid API Response:")
    print(json.dumps(data, indent=2))
    
except ContractViolationError as e:
    print(f"❌ Schema validation failed: {e}")

In [ ]:
# Example 2: Content Moderation Pipeline
print("🛡️ Example 2: Content Moderation Pipeline\n")

# LLMCL contract for content moderation
moderation_contract = """
contract ContentModeration(priority = critical) {
    # Input checks
    require not match(content, "(?i)(hate|violence|harassment)")
        message: "Inappropriate content detected in input"
    
    # Output checks
    ensure not contains(response, "offensive")
        message: "Response contains inappropriate content"
    
    ensure contains(response, "respectful") or contains(response, "appropriate")
        message: "Response should emphasize respectful communication"
        auto_fix: "I appreciate your question. Let me provide a respectful and appropriate response."
}
"""

# Load and test moderation contract
async def test_moderation():
    mod_name = await runtime.load_contract(moderation_contract)
    mod_context = runtime.create_context("moderation_session")
    runtime.add_contract_to_context("moderation_session", mod_name)
    
    # Test appropriate content
    appropriate = "How can I help my community?"
    result = await runtime.validate(
        appropriate,
        "moderation_session",
        validation_type="input",
        additional_context={"content": appropriate}
    )
    print(f"✅ Appropriate content: {result.is_valid}")
    
    # Test inappropriate content
    inappropriate = "I hate this service"
    result = await runtime.validate(
        inappropriate,
        "moderation_session",
        validation_type="input",
        additional_context={"content": inappropriate}
    )
    print(f"\n❌ Inappropriate content: {result.is_valid}")
    if not result.is_valid:
        print(f"   Reason: {result.message}")

await test_moderation()

In [ ]:
# Example 3: Multi-turn Conversation with State Tracking
print("💬 Example 3: Multi-turn Conversation Management\n")

# LLMCL contract for conversation management
conversation_contract = """
contract ConversationManagement {
    # Track conversation state
    state user_name: string = ""
    state topic: string = ""
    state turn_count: int = 0
    
    # Temporal constraints
    temporal within 3 extract(response, "name") -> user_name
        message: "Should ask for user's name within first 3 turns"
    
    # Consistency checks
    ensure turn_count > 3 implies len(user_name) > 0
        message: "Should have user's name by turn 4"
    
    # Quality metrics
    ensure_prob contains(response, user_name) when len(user_name) > 0, 0.7
        message: "Should use user's name in 70% of responses"
        window_size: 10
}
"""

# Create conversation provider
conv_provider = ImprovedOpenAIProvider(api_key=api_key)
conv_provider.add_output_contract(ConversationConsistencyContract())

print("Starting multi-turn conversation...\n")

# Simulate conversation
messages = [
    {"role": "system", "content": "You are a helpful assistant. Start by greeting the user and asking their name."}
]

# Turn 1
response = conv_provider.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages,
    max_tokens=100
)
assistant_message = response.choices[0].message.content
print(f"Assistant: {assistant_message}")
messages.append({"role": "assistant", "content": assistant_message})

# Turn 2 - User provides name
user_message = "Hi! My name is Alice."
print(f"\nUser: {user_message}")
messages.append({"role": "user", "content": user_message})

response = conv_provider.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages,
    max_tokens=100
)
assistant_message = response.choices[0].message.content
print(f"Assistant: {assistant_message}")

print("\n✅ Multi-turn conversation with consistency tracking")

In [ ]:
# Example 4: Medical/Legal Disclaimer Enforcement
print("⚕️ Example 4: Medical Disclaimer Enforcement\n")

# Create provider with medical disclaimer contract
medical_provider = ImprovedOpenAIProvider(api_key=api_key)
medical_provider.add_output_contract(MedicalDisclaimerContract())

try:
    response = medical_provider.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": "I have a headache. What medication should I take?"}
        ]
    )
    
    content = response.choices[0].message.content
    print("Response with medical disclaimer:")
    print(content)
    
    # Check if disclaimer is present
    disclaimer_keywords = ["medical advice", "healthcare professional", "doctor", "physician"]
    has_disclaimer = any(keyword in content.lower() for keyword in disclaimer_keywords)
    
    if has_disclaimer:
        print("\n✅ Medical disclaimer properly included")
    else:
        print("\n⚠️ Medical disclaimer may be missing")
        
except ContractViolationError as e:
    print(f"❌ Contract violation: {e}")

## Summary

This notebook demonstrated the key features of the OpenAI Python SDK with LLM Contract Language framework:

### Key Features Covered:
1. **Contract-Based Validation**: Input and output validation with multiple contract types
2. **LLMCL**: Domain-specific language for defining contracts declaratively
3. **Streaming Support**: Real-time validation of streaming responses
4. **Auto-Remediation**: Automatic fixing of contract violations
5. **Performance Monitoring**: Comprehensive metrics and health reporting
6. **Circuit Breaker**: Fault tolerance for production environments
7. **Async Support**: Full async/await compatibility
8. **Custom Contracts**: Extensible framework for custom validation logic

### Benefits:
- **Reliability**: Ensure LLM outputs meet specific requirements
- **Security**: Detect and prevent prompt injection attacks
- **Compliance**: Enforce content policies and legal requirements
- **Performance**: Monitor and optimize contract validation
- **Developer Experience**: Drop-in replacement for OpenAI SDK

### Next Steps:
1. Explore more contract types in `llm_contracts/contracts/base.py`
2. Create custom contracts for your specific use cases
3. Integrate with your production applications
4. Monitor metrics and optimize performance
5. Contribute to the open-source project

For more information, see the documentation in the `docs/` directory.